# Introduction librairies sklearn, pandas
- Pandas : manipuler des fichiers csv (tableaux excel), faire des statistiques descriptives (moyenne variance, mediane,histogramme) 
- Sklearn : Librairie orienté machine learning. Simplifie les taches récurentes, telles que séparer les données, créer un modèle, entraîner un modèle.


La librairie  "pandas" est destinée à utiliser des fichiers csv sous python de manière naturelle. Les fichiers csv sont représentés par un "DataFrame". C'est un objet python sur lesquels on peut effectuer les opérations les plus courantes (calculer la moyenne d'une colonne, etc.)

# Importer les données
Afficher les colonnes :

In [ ]:
import pandas
import matplotlib.pyplot as plt
df=pandas.read_csv("housing.csv")
df.columns

Le "Boston data frame" a 506 lignes and 14 colonnes.

- crim taux de criminalité par ville

- zn proportion de résidences ( lots > 25,000 sq.ft.)

- indus proportion  de zone industrielle par ville

- chas 1 si une rivière passe, 0 sinon

- nox nitrogen oxides concentration (pollution).

- rm nombre moyen de chambres par habitation

- age proportion de maisons occupées construites avant 1940.

- dis distance moyenne (pondérée) des 5 centres d'emplois de Boston.

- rad indice d'accés au périph (plus grand: accès plus rapide au périph)

- tax taxe d'habitation  pour  \$10,000.

- ptratio taux de "pupil-teacher" par ville (?).

- black 1000(Bk - 0.63)^2 ou Bk est la proportion d'afro-américain par ville (base de donnée US)

- lstat pourcentage de personnes en dessous du seuil de pauvreté (lower status).


# Outils de visualisation

## Aperçu des données

In [ ]:
df.head()

# Visualisation des données : Statistiques descriptives
Avec pandas et l'objet "Dataframe", il est facile de visualiser la distribution des variables présentes dans notre jeu de donnée

In [ ]:
df.hist(figsize=(30,10))

## Selectionner une colone

In [ ]:
age = df.age
print(age)

## statistiques sur une colonne


In [ ]:
max_age=age.max()
min_age=age.min()
median_age=age.median()
print(max_age,min_age,median_age)

## Visualiser l'impact d'une variable sur une autre
- Faire le scatter plot de crim contre medv

In [ ]:
df.plot.scatter(x="crim", y="medv",alpha=0.5)


A faire : essayer de trouver d'autres variables importantes qui pourraient expliquer medv

# Partie 2


# Séparer sous la forme (X,Y)

In [ ]:
y=df["medv"] # la variable a prédire
del df["medv"] # enlever la variable à prédire des prédicteurs
x=df
print("la taille de mon jeu de donnée est {}. Je cherche à expliquer le prix de vente d'une maison à partir de {} variables".format(x.shape[0],x.shape[1]))

In [ ]:
x=x.fillna(-1) # valeurs manquantes
y=y.fillna(-1) # valeurs manquantes

# Création du jeu d'entrainement et du jeu de test
Nous devons maintenant découper notre jeu d'entrainement. Pour cela, j'utilise la fonction train_test_split fournie par sklearn. Le paramètre test_size me permet de choisir la proportion de mon jeu de test. En règle général, cette valeur est comprise entre 0.2 et 0.3. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2)

# Création d'un modèle (regréssion lineaire)
Nous allons maintenant essayer d'expliquer la variable y (le prix de la maison) en fonction des variables x. Le modèle est de la forme : $$y=A*x+b$$
$$ Prix=\beta_{CRIM}*X_{CRIM}+\beta_{ZN}*X_{ZN}+\beta_{INDUS}*X_{INDUS}+...$$
Où les coefficients du vecteur A seront ceux qui "expliquent" le mieux les données. Par exemple, on pourrait faire l'hypothèse que la variable "CRIM" (taux de criminalité de la zone) a un impact négatif sur le prix des maisons. On peut prévoir un coefficient négatif devant cette variable.

In [ ]:
import sklearn.linear_model

model=sklearn.linear_model.LinearRegression()
model.fit(X_train,Y_train) # Attention ! Il ne faut surtout pas mettre les données de test dans la fonction fit

In [ ]:
df.columns

In [ ]:
model.coef_

-Interpréter la valeur de ces coefficients. Sont ils crédibles ?

In [ ]:
for i in range(len(df.columns)):
    print("Le coefficient de {} est {}".format(df.columns[i],model.coef_[i]))

# Evaluation du modèle
Comment savoir si notre modèle est bon ou pas ? Il faut un critère de comparaison. Par exemple l'écart quadratique moyen pour une regression, le taux de bonnes prédictions pour une classification

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
prediction_train=model.predict(X_train)
erreur_train=mean_squared_error(Y_train,prediction_train)
print(erreur_train)

Et sur le jeu d'entrainement ?

In [ ]:
prediction_test=model.predict(X_test)
erreur_test=mean_squared_error(Y_test,prediction_test)
print(erreur_test)

L'objectif est le suivant : avoir la plus petite erreur possible sur le jeu d'entrainement. 

### Interpréter l'erreur
cette erreur est elle acceptable ? Notre modèle est il bon ? Nous avons ici considéré l'erreur moyenne au carré, peut être qu'il faut scorer le modèle avec une erreur plus compréhensible.

In [ ]:
erreur_train=mean_absolute_error(Y_test,prediction_test)
print(erreur_train)

La contrainte est la suivante : avoir une erreur similaire sur le jeu de test (contrôle d'overfitting)

# Visualisation des résultats

In [ ]:
import matplotlib.pyplot as plt
erreurs=Y_test-prediction_test

In [ ]:
plt.figure()
plt.hist(erreurs, bins=50)
plt.show()


- Comment calculer le biais ?
- Analyser l'histogramme des erreurs 

In [ ]:
erreurs.mean() # bais de 2295